In [1]:
import json
import time
import glob
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy.polynomial.chebyshev as cheb
from scipy.stats import moment
from itertools import permutations
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import clear_output
from scipy.interpolate import lagrange


In [147]:
timeout = 3
LAMBDA = 0.01
MU = 100
SIGMA = 40
VARK = 1

def r(task):
    ind = 0
    if isinstance(task, TaskSet):
        return task.array[ind]
    return task[ind]

def p(task):
    ind = 1
    if isinstance(task, TaskSet):
        return task.array[ind]
    return task[ind]

def d(task):
    ind = 2
    if isinstance(task, TaskSet):
        return task.array[ind]
    return task[ind]

def remove(arr, elem):
    return np.delete(arr, np.where(np.in1d(arr, elem)))

class TaskSet:
    
    def __init__(self, a):
        if isinstance(a, int):
            rs = np.cumsum(np.random.exponential(scale=1/LAMBDA, size=(a,)))
            ps = np.clip(np.random.normal(MU, SIGMA, size=(a,)), a_min=0, a_max=None)
            ds = [r + VARK*moment(ps, moment=2) for r in rs]
            self.array = np.array([rs, ps, ds]).T.astype(float)
        else:
            self.array = np.copy(a)
            
    def __repr__(self):
        return "  r  |  p  |  d  \n" + str(self.array)
    
    def copy(self):
        return TaskSet(self.array)
    
    def __getitem__(self, key):
        return TaskSet(self.array[key])
    
    def __iter__(self):
        return iter(self.array)
    
    def C(self, i, tau=0):
        t = tau
        for task in self.array[:i+1]:
            if t < r(task): t = r(task)
            t += p(task)
        return t
    
    def C_max(self, tau=0):
        t = tau
        for task in self.array:
            if t < r(task): t = r(task)
            t += p(task)
        return t
    
    def L(self, i=None, tau=0):
        if i is None:
            return self.C_max(tau) - d(self[-1])
        return self.C(i, tau) - d(self[i])
    
    def L_max(self, tau=0):
        if len(self) == 0: return float('inf')
        return max([self.L(i, tau) for i, _ in enumerate(self)])
    
    def T(self, i=None, tau=0):
        return max(0, self.L(i, tau))
    
    def T_max(self, tau=0):
        return max(0, self.L_max(tau))
    
    def __len__(self):
        return len(self.array)
    
    def __eq__(self, other):
        return self.array == other
    
    def without(self, indexes):
        return TaskSet(np.delete(self.array, np.array(indexes).astype(float), axis=0))
    
    def find(self, item):
        return np.where((self.array == item).all(axis=1))[0]
    
    def transpose(self):
        return self.array.T
    
    def scale(self, alpha, key='r'):
        if key == 'r': ind = 0
        elif key == 'p': ind = 1
        else: ind = 2
        self.array[:,ind] = self.array[:,ind]*alpha
        return self
    
def dual(N, tau, B):
    if len(N.without(B)) == 0: return float('inf')
    pi_r = r(np.argsort(N, axis=0).transpose())
    bestL = N[pi_r].L(tau=tau)
    for i_k in pi_r:
        toDrop = B.copy()
        toDrop.append(i_k)
        #print(toDrop)
        s = N.without(toDrop)
        #print(s)
        if len(s) != 0:
            task_l = min(s, key=r)
            i_l = N.find(task_l)[0]
            pi_k = remove(pi_r, [i_l, i_k])
            pi_k = np.insert(pi_k, 0, i_l)
            pi_k = np.append(pi_k, i_k)
            L_k = N[pi_k].L(tau=tau)
            if L_k < bestL:
                bestL = L_k
    additionalL = N[pi_r].L(i=0, tau=tau)
    if additionalL > bestL:
        bestL = additionalL
    return bestL

class Instance:
    
    def __init__(self, N, tau=0, pi=[], B=[]):
        self.N = N.copy()
        self.tau = tau
        self.pi = pi.copy()
        self.B = B.copy()
        self.nu = dual(N, tau, B)
        
    def __getitem__(self, key):
        return TaskSet(self.N.array[key])
        
    def best_job(self):
        s = self.N.without(self.B)
        sn = s[r(s.transpose()) <= self.tau]
        if len(sn) == 0:
            f = min(s, key=r)
            #self.tau = r(f)
            #self.nu = dual(self.N, self.tau, self.B)
        else:
            f = min(sn, key=d)
        return self.N.find(f)[0]
    
    def L(self, i=None):
        return self[self.pi].L(i, self.tau)
    
    def T(self, i=None):
        return self[self.pi].T(i, self.tau)
        
    def L_max(self):
        return self[self.pi].L_max(self.tau)
    
    def T_max(self):
        return self[self.pi].T_max(self.tau)
    
    def __repr__(self):
        return "Instance:\n" + repr(self.N) + "\nnu  = " + str(self.nu) + "\ntau = " + str(self.tau) + "\npi  = " + str(self.pi) + "\nB   = " + str(self.B)
    
    
def main(N, tau=0, verbose=False, modified=False):
    tb = time.time()
    b_counter = 0
    #print("bi")
    instances = [Instance(N, tau)]
    #print("ai")
    if modified: bestPi = list(range(len(N)))
    else: bestPi = []
    while len(instances) > 0:
        ti = time.time()
        if ti - tb > timeout:
            return TaskSet([]), -1
        bestInstanceIndex, bestInstance = min(enumerate(instances), key=lambda x: x[1].nu) # + N[x[1].pi].L_max(tau))
        instances.pop(bestInstanceIndex)
        f = bestInstance.best_job()
        f_data = bestInstance[f]
        N1 = bestInstance.N.without(f)
        tau1 = max(r(f_data), bestInstance.tau) + p(f_data)
        B1 = []
        pi1 = bestInstance.pi.copy()
        pi1.append(N.find(f_data)[0])
        i1 = Instance(N1, tau1, pi1, B1)
        N2 = bestInstance.N
        tau2 = bestInstance.tau
        B2 = bestInstance.B.copy()
        B2.append(N2.find(f_data)[0]) #!
        pi2 = bestInstance.pi
        i2 = Instance(N2, tau2, pi2, B2)
        instances += [i1, i2]
        b_counter += 1
        #print(i1)
        if len(pi1) == len(N):
            #print(N[bestPi].L_max(tau))
            #print(pi1)
            if N[pi1].L_max(tau) < N[bestPi].L_max(tau):
                bestPi = pi1.copy()
                if verbose: print(bestPi, '\tLmax =', N[bestPi].L_max(tau))
        #lb = len(instances)
        instances = [i for i in instances if max(i.nu, N[i.pi].L_max(tau)) < N[bestPi].L_max(tau)]
        #print(lb, len(instances))
    return bestPi, b_counter
        

In [148]:
def lagrange_interpolation(key, s):
    alphaRange = np.arange(0, 2.1, 0.2)
    scaledSchedules = []
    for alpha in alphaRange:
        scaledSchedules.append(s.copy().scale(alpha, key=key))
    flattenedScaledSchedules = list(map(lambda x: x.array.flatten(), scaledSchedules))

    #print("\rCurrently working with", n)
    approxResults = []
    if len(alphaRange) != len(scaledSchedules): raise RuntimeError()
    for i, a in enumerate(alphaRange):
        if a != 1:
            s = scaledSchedules[i]
            sched, count = main(s, verbose=False)
            if count == -1: continue
            Lmax = s[sched].L_max()
            approxResults.append((a, Lmax))
    approxResults = np.array(approxResults)
    x = approxResults[:,0]
    y = approxResults[:,1]
    #print(len(x))
    #print(x, y)
    poly = lagrange(x, y)
    return poly(1.)

def chebyshev_interpolation(k, schedule):
    global realResult
    global approxResults
    global m
    global s
    global key
    key = k
    s = schedule

    def test_schedule(alpha_arr):
        global key
        global approxResults
        global m
        global s
        approxResults = []
        for a in alpha_arr:
            #print("\ra =", a, end='')
            scaled_s = s.copy().scale(a, key=key)
            #flattenedScaled_s = list(map(lambda x: x.array.flatten(), scaled_s))
            bestPi, bestCounter = main(scaled_s, verbose=False)
            Lmax = scaled_s[bestPi].L_max()
            approxResults.append((a, Lmax))
        approxResults = np.array(approxResults)
        return approxResults[:,1]

    
    C = cheb.Chebyshev.interpolate(test_schedule, 13, [0, 2])
    return C(1.)


def mul_lagrange_interpolation(s):
    N_POINTS = 10

    delta_r = 0.1
    delta_p = 0.2
    delta_d = 0.05
    N_TASKS = 3

    scaledSchedules = []
    alphaRange = []
    #alphaRange.append(1.)
    ticks = [(1.,1.,1.)]
    for factor in range(N_POINTS):
        factor_r = (5+factor)*delta_r
        factor_p = factor*delta_p
        factor_d = (15+factor)*delta_d
        if factor_r != 1:
            scaledSchedules.append(s.copy().scale(factor_r, key='r')
                                   .scale(factor_p, key='p')
                                   .scale(factor_d, key='d'))
            alphaRange.append(factor_r)
            ticks.append((round(factor_r,2),
                      round(factor_p,2),
                      round(factor_d,2)))
    flattenedScaledSchedules = list(map(lambda x: x.array.flatten(), scaledSchedules))
    #print(ticks)
    #print("\rCurrently working with", n)
    approxResults = []
    if len(alphaRange) != len(scaledSchedules): raise RuntimeError()
    for i, a in enumerate(alphaRange):
        s = scaledSchedules[i]
        sched, count = main(s, verbose=False)
        if count == -1: continue
        Lmax = s[sched].L_max()
        #print(n, alphaRange[i], sched)
        approxResults.append((a, Lmax))
    approxResults = np.array(approxResults)
    x = approxResults[:,0]
    y = approxResults[:,1]
    #print(len(x))
    #print(x, y)
    poly = lagrange(x, y)
    return poly(1.)


In [124]:
s[pi].L_max()

-264.3352907046266

In [117]:
lagrange_interpolation('r', s)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


-9.175864657424142

In [92]:
N_TASKS = 250
N_JOBS = 8

tasks = []
for i in range(N_TASKS):
    s = TaskSet(N_JOBS)
    tasks.append(s)
        
with open('data.pickle', 'wb') as f:
    pickle.dump(tasks, f)

In [149]:
with open('data.pickle', 'rb') as f:
    tasks = pickle.load(f)

In [150]:
df = pd.DataFrame(columns=['Complexity',
                           'L_max',
                           'r',
                           'p',
                           'd',
                           'Multi',
                           #'r_Cheb',
                           #'p_Cheb',
                           #'d_Cheb',
                           'Orig time',
                           'r time',
                           'Multi time',
                           'Schedule'])
pBar = IntProgress(min=0, max=len(tasks))
display(pBar)
for i, s in enumerate(tasks):
    tb = time.time()
    pi, count = main(s)
    to = time.time() - tb
    if count == -1:
        continue
    Lmax = s[pi].L_max()
    
    tb = time.time()
    r_lagrange = lagrange_interpolation('r', s)
    tr = time.time() - tb
    
    p_lagrange = lagrange_interpolation('p', s)
    d_lagrange = lagrange_interpolation('d', s)
    
    tb = time.time()
    m_lagrange = mul_lagrange_interpolation(s)
    tm = time.time() - tb
    #r_chebyshev = chebyshev_interpolation('r', s)
    #p_chebyshev = chebyshev_interpolation('p', s)
    #d_chebyshev = chebyshev_interpolation('d', s)
    df.loc[i] = [count,
                 Lmax,
                 r_lagrange,
                 p_lagrange,
                 d_lagrange,
                 m_lagrange,
                 #r_chebyshev,
                 #p_chebyshev,
                 #d_chebyshev,
                 to,
                 tr,
                 tm,
                 pi]
    pBar.value += 1
df.to_excel("sample.xlsx")

IntProgress(value=0, max=250)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


ZeroDivisionError: float division by zero